In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!nvidia-smi

Wed Jul 24 01:38:16 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P0             25W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!rm -rf /kaggle/working/logs

In [3]:
!mkdir /kaggle/working/logs
!mkdir /kaggle/working/logs/weights
!ls /kaggle/working/logs

weights


In [ ]:
!git clone https://github.com/lamnguyenvu98/YOWOv2-PL.git
!ls /kaggle/working/

In [2]:
%cd YOWOv2-PL

/kaggle/working/YOWOv2-PL


In [3]:
!git pull

Already up to date.


In [3]:
%%capture
!pip install .

In [4]:
# only for wandb
# click on Add-ons -> Secret -> Add secret
# provide LABEL as wandb_api_key
# VALUE as <your_wandb_api_key>
import wandb
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

my_secret = user_secrets.get_secret("wandb_api_key") 

wandb.login(key=my_secret)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [8]:
# run = wandb.init(
#     project="spatial-temporal-action-detection-yowov2",
#     id="i2yvhgrp"
# )
# download_path = run.use_model(
#     name="lamnguyen-vu98-eratodev/spatial-temporal-action-detection-yowov2/model-i2yvhgrp:v2"
# )
# print(download_path)

wandb: Currently logged in as: lamnguyen-vu98 (lamnguyen-vu98-eratodev). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact model-i2yvhgrp:v2, 125.48MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:2.7


/kaggle/working/YOWOv2-PL/artifacts/model-i2yvhgrp:v2/model.ckpt


In [ ]:
# !rm /kaggle/working/logs/config.yaml

In [8]:
!yowo fit -c configs/yowov2_tiny_ucf24_jhmdb21.yaml \
    --seed_everything=42 \
    --data.data_dir="/kaggle/input/ucf24-spatial-temporal-localization-yowo/ucf24" \
    --data.batch_size.train=64 \
    --data.batch_size.val=32 \
    --data.num_workers="auto" \
    --model.warmup_config.scheduler.max_iter=250 \
    --model.warmup_config.interval="step" \
    --model.freeze_backbone_2d="false" \
    --model.freeze_backbone_3d="false" \
    --model.model_config.head_act="silu" \
    --model.model_config.use_blurpool="true" \
    --trainer.max_epochs=7 \
    --trainer.accumulate_grad_batches=2 \
    --trainer.accelerator="gpu" \
    --trainer.devices=1 \
    --trainer.precision="16-mixed" \
    --trainer.strategy="auto" \
    --trainer.callbacks+="lightning.pytorch.callbacks.ModelCheckpoint" \
    --trainer.callbacks.monitor="map" \
    --trainer.callbacks.filename="{epoch}-{map:.3f}" \
    --trainer.callbacks.mode="max" \
    --trainer.callbacks.save_last="true" \
    --trainer.callbacks.save_top_k=3 \
    --trainer.callbacks.dirpath="/kaggle/working/logs/weights" \
    --trainer.logger.class_path="lightning.pytorch.loggers.WandbLogger" \
    --trainer.logger.name="experiment-yowo-tiny_2024-07-22_00:35" \
    --trainer.logger.project="spatial-temporal-action-detection-yowov2" \
    --trainer.logger.tags="['P100', 'ucf24', 'tiny', 'freeze-backbone', 'blurpool']" \
    --trainer.logger.log_model="all" \
    --trainer.logger.save_dir="/kaggle/working/logs" \
    --trainer.logger.id="i2yvhgrp" \
    --ckpt_path="/kaggle/working/logs/weights/last.ckpt"
#     --trainer.check_val_every_n_epoch=1 \
#     --trainer.default_root_dir="/kaggle/working/lightning_logs" \
#     --trainer.logger.id="ycd10izz"
#     --trainer.callbacks.dirpath="/kaggle/working/checkpoints" \
#     --trainer.logger.save_dir="/kaggle/working/logs" \
#     --trainer.callbacks.every_n_train_steps=500 \
#     --ckpt_path="/kaggle/working/checkpoints/last-v1.ckpt" \
#     --ckpt_path="/kaggle/working/checkpoints/last.ckpt" \
#     --trainer.logger.entity="nguyen" \
#     --trainer.logger.class_path="lightning.pytorch.loggers.CSVLogger" \ # add logger
#     --trainer.logger.init_args.save_dir="/kaggle/working/logs" \
#     --trainer.logger.init_args.name="yowov2_free_nano" \
#     --ckpt_path="/kaggle/working/work_dir/last.ckpt" \

INFO: Seed set to 42
FPN: pafpn_elan
Head: Decoupled Head
Head: Decoupled Head
Head: Decoupled Head
Loading 2D backbone pretrained weight: YOLO_FREE_TINY
Loading pretrained weight ...
Loading 3D backbone pretrained weight: SHUFFLENET_V2_X2_0
conv_last.0.weight
conv_last.1.weight
conv_last.1.bias
conv_last.1.running_mean
conv_last.1.running_var
conv_last.1.num_batches_tracked
classifier.1.weight
classifier.1.bias
Head: Decoupled Head
Head: Decoupled Head
Head: Decoupled Head
INFO: Using 16bit Automatic Mixed Precision (AMP)
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
wandb: Currently logged in as: lamnguyen-vu98 (lamnguyen-vu98-eratodev). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.0
wandb: Run data is saved locally in /kaggle/working/logs/wandb/run-2

In [ ]:
# Validation
!yowo validate -c configs/yowov2_tiny_ucf24_jhmdb21.yaml \
    --seed_everything=42 \
    --data.data_dir="/kaggle/input/ucf24-spatial-temporal-localization-yowo/ucf24" \
    --data.batch_size.val=64 \
    --data.num_workers="auto" \
    --model.freeze_backbone_2d="false" \
    --model.freeze_backbone_3d="false" \
    --model.model_config.head_act="silu" \
    --model.model_config.use_blurpool="true" \
    --trainer.accelerator="gpu" \
    --trainer.devices=1 \
    --trainer.precision="16-mixed" \
    --ckpt_path="/kaggle/working/logs/weights/last.ckpt" \
    --trainer.fast_dev_run=10

In [ ]:
!ls -l /kaggle/working/lightning_logs

In [ ]:
!mv /kaggle/working/checkpoints/last-v1.ckpt /kaggle/working/checkpoints/last.ckpt

In [ ]:
import torch
import os
path = "/kaggle/working/checkpoints"
ckp = torch.load(os.path.join(path, "last-v1.ckpt"), map_location=torch.device("cpu"))
ckp['epoch'], ckp['global_step']

In [ ]:
!cp /kaggle/working/checkpoints/last-v1.ckpt /kaggle/working/

In [ ]:
from IPython.display import FileLink
FileLink('last-v1.ckpt')